In [ ]:
import os
import requests
import pandas as pd
import numpy as np

alphavantage_api_key = open("alphavantage_api_key.txt").read()
ticker = "amat".upper()

# Calls the API to get historical stock data
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=full&apikey={alphavantage_api_key}'
r = requests.get(url)
data = r.json()

# Calls the API to get earnings data
url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={alphavantage_api_key}'
r = requests.get(url)
earnings = r.json()

# Yahoo Finance API to get earnings dates
#yfinance_earnings = yf.Ticker(f'{ticker}').get_earnings_dates(limit=30)


In [ ]:
##############
# Stock Data #
##############

# Extract the time series data
time_series = data['Time Series (Daily)']

# Convert the time series data into a DataFrame
df = pd.DataFrame.from_dict(time_series, orient='index')

# Convert the index to datetime
df.index = pd.to_datetime(df.index)

# Convert the columns to numeric
df = df.apply(pd.to_numeric)

df = df.reset_index().rename(columns={'index': 'tradingDate'})

#################
# Earnings Data #
#################

# Extract annual earnings data
annual_earnings = earnings['annualEarnings']

# Convert annual earnings data into a DataFrame
annual_earnings_df = pd.DataFrame(annual_earnings)

# Extract quarterly earnings data
quarterly_earnings = earnings['quarterlyEarnings']

# Convert quarterly earnings data into a DataFrame
quarterly_earnings_df = pd.DataFrame(quarterly_earnings)

last_earnings = quarterly_earnings_df[["reportedDate","reportTime"]].head(30)

# Function to adjust the date based on reportTime
def adjust_date(row):
   if row['reportTime'] == 'post-market':
       return row['reportedDate'] + pd.Timedelta(days=1)
   elif row['reportTime'] == 'pre-market':
       return row['reportedDate'] - pd.Timedelta(days=1)
   else:
       return row['reportedDate']

last_earnings['reportedDate'] = pd.to_datetime(last_earnings['reportedDate'])

# Apply the function to create a new column with adjusted dates
# Last earnings is used to determine how the stock moved historically around earnings
last_earnings['adjustedDate'] = last_earnings.apply(adjust_date, axis=1)

#############
# merged_df #
#############

#merged_df joins the first set of earnings data
merged_df = pd.merge(last_earnings, df, left_on='reportedDate', right_on="tradingDate", how='left')
columns_to_drop = ['tradingDate','5. volume']
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df = merged_df.rename(columns={
   '1. open': 'report open',
   '2. high': 'report high',
   '3. low': 'report low',
   '4. close': 'report close'
})

###############
# merged_2_df #
###############

#merged_2_df is the final joined dataset that combines the first set of earnings data with the second set of earnings data
merged_2_df = pd.merge(merged_df, df, left_on='adjustedDate', right_on="tradingDate", how='left')

merged_2_df.drop(columns=columns_to_drop, inplace=True)

merged_2_df = merged_2_df.rename(columns={
   '1. open': 'adjusted open',
   '2. high': 'adjusted high',
   '3. low': 'adjusted low',
   '4. close': 'adjusted close'
})

################
# calculations #
################

# used to calculate the difference between the high and low depending is earnings was pre-market or post-market
# and if the stock went up or down

# Initialize a new column with NaN values to calculate the difference between high and low
merged_2_df['diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   #Post-market
   if row['reportTime'] == 'post-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted low'] - row['report high'],2)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted high'] - row['report low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan
   #Pre-market
   elif row['reportTime'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report low'] - row['adjusted high'],2)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report high'] - row['adjusted low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan

# Initialize a new column with NaN values for the percentage calculation
merged_2_df['percent_diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   if row['reportTime'] == 'post-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round((row['adjusted low'] / row['report high'])-1,3)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['adjusted high'] / row['report low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan
   elif row['reportTime'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report low'] / row['adjusted high']-1,3)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report high'] / row['adjusted low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan

merged_2_df
